Based on demonstration of maximum-likelihood reconstruction with SIRF

# A notebook to get you used to some of the syntax of SIRF and PyTorch and introduce the ideas behind the sensitivity image and why we'd want to change it

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib inline

# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'ML_reconstruction')

In [ ]:
#%% Initial imports etc
import numpy as np
import scipy.stats
from scipy import ndimage
from scipy.ndimage import affine_transform
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import sys
import shutil
from numba import njit, prange
#%% Read in images
import brainweb
from tqdm import tqdm
#import scipy
#from scipy import optimize
import sirf.STIR as pet
import sirf.Reg as reg
from sirf.Utilities import examples_data_path
from sirf_exercises import exercises_data_path
pet.set_verbosity(0)

# increase dpi of matplotlib figures
plt.rcParams['figure.dpi'] = 200

# define the directory with input files for this notebook
data_path = os.path.join(examples_data_path('PET'), 'thorax_single_slice')

In [ ]:
# set-up redirection of STIR messages to files
msg_red = pet.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')

In [ ]:
#%% some handy function definitions
def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar(shrink=.4)
    plt.title(title)
    plt.axis("off")

def make_positive(image_array):
    """truncate any negatives to zero"""
    image_array[image_array<0] = 0
    return image_array

def make_cylindrical_FOV(image):
    """truncate to cylindrical FOV"""
    filter = pet.TruncateToCylinderProcessor()
    filter.apply(image)

## Create some simulated data from ground-truth images

We'll use brainweb's data base to simulate brain FDG-18 images here. Don't worry too much about this next part

In [ ]:
fname, url= sorted(brainweb.utils.LINKS.items())[0]
files = brainweb.get_file(fname, url, ".")
data = brainweb.load_file(fname)

brainweb.seed(1337)

for f in tqdm([fname], desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(f, petNoise=1, t1Noise=0.75, t2Noise=0.75, petSigma=1, t1Sigma=1, t2Sigma=1)

FDG_arr  = vol['PET']
T1_arr   = vol['T1']
uMap_arr = vol['uMap']

In [ ]:
def crop(img, cropx, cropy):
    z,y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[:, starty:starty+cropy,startx:startx+cropx]

# function to zoom out and crop the brainweb images
fdg_zoomed = crop(ndimage.zoom(FDG_arr, 1, order=1), 155, 155) # crop and zoom image
fdg_zoomed = np.expand_dims(fdg_zoomed[50,:,:], axis = 0) # expand for pseudo 3D

umap_zoomed = crop(ndimage.zoom(uMap_arr, 1, order=1), 155, 155) # crop and zoom image
umap_zoomed = np.expand_dims(umap_zoomed[50,:,:], axis = 0) # expand for pseudo 3D

image_template = pet.ImageData(os.path.join(data_path, 'emission.hv'))
image = image_template.clone().fill(fdg_zoomed)
attn_image = image_template.clone().fill(umap_zoomed)

template = pet.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))

plt.figure()
plot_2d_image([1,2,1], image.as_array()[0,:], "image")
plot_2d_image([1,2,2], attn_image.as_array()[0,:], "attenuation image")
plt.show()

In [ ]:
#%% save max for future displays
cmax = image.max()*.6

We are going to start creating our detector model. A PET (or SPECT or CT or...) acquisition process is characterised by a system matrix, $\mathcal{A}$, as well as additive contributions consisting of scatter and random coincidences:
$$f = \mathcal{A}u + s + r $$
Wwhere $f$ is our data, $u$ is our image and $s,r$ are our additive componatnts. We will only model the system matrix in this notebook.

Now, our system model is itself comprised of a number of different operations. We will concentrate on three of these: the radon transform, $\mathcal{R}$, attenuation, $A$, and detector normalisation $\mathcal{N}$, giving us:
$$ \mathcal{A} \approx \mathcal{N} A \mathcal{R} $$
where attenuation corrections and normalisation and multiplicative factors in the projection domain. \
And so our next job is to use SIRF's software to build this acquisition model

In [ ]:
# create acquisition model matrix (this uses something called ray tracing, which you can ignore for this example)
# This is a 3-D (or in our case 2-D) Radon Transform matrix
acq_model_matrix  = pet.RayTracingMatrix()
# we will increase the number of rays used for every Line-of-Response (LOR) as an example
# (it is not required for the exercise of course)
acq_model_matrix.set_num_tangential_LORs(5)
# We can now create the acquisition model using this matrix
acq_model = pet.AcquisitionModelUsingMatrix(acq_model_matrix)

And we have $\mathcal{A} \approx \mathcal{R} $ 

In [ ]:
# We will now create the acquisition sensitivity model - a sinogram containing the sensitivity of each LOR. T
# This will depend on individual detector efficiencies, the geometry of the scanner, and the attenuation image
acq_model_for_attn = pet.AcquisitionModelUsingRayTracingMatrix() # this saves us a line of code but is the same as in the previous cell
# We now create the sensitivity model using the acquisition model and the attenuation image
asm_attn = pet.AcquisitionSensitivityModel(attn_image, acq_model_for_attn)
asm_attn.set_up(template)
# we can now find the attenuation sensitivity factors for each LOR by forward projecting a uniform image.
# We can set the value of this uniform image to be our detector efficiency. For now, let's just use 1.
attn_factors = asm_attn.forward(template.get_uniform_copy(1))
plt.figure()
plot_2d_image([1,1,1], attn_factors.as_array()[0,0,:], "LOR attenuation sensitivity factors")

This "image" looks a bit funny. Hopefully you've done some reading into this already, but this is what's know as a sinogram (because of the sinusoidal shape) and consists of 2D views of the object from different angles stacked on top of eachother. It's basicall what our detectors 'see' from a uniform image of 1.

In [ ]:
# And then add the detector sensitivity (based on the attenuation image) that we made previously
acq_model.set_acquisition_sensitivity(asm_attn)
# set-up
acq_model.set_up(template,image)

And we now have $\mathcal{A} \approx \mathcal{N} A \mathcal{R} $. 

We can then see how this system will forward project our image and use this to simulate some data

In [ ]:
#%% simulate some data using forward projection
acquired_data=acq_model.forward(image)
plot_2d_image([1,1,1], acquired_data.as_array()[0,0,:], "Acquired data")

In [ ]:
def add_noise(proj_data,noise_factor = 0.1, seed = 50):
    """Add Poission noise to acquisition data."""
    proj_data_arr = proj_data.as_array() / noise_factor
    # Data should be >=0 anyway, but add abs just to be safe
    np.random.seed(seed)
    noisy_proj_data_arr = np.random.poisson(proj_data_arr).astype('float32');
    noisy_proj_data = proj_data.clone()
    noisy_proj_data.fill(noisy_proj_data_arr*noise_factor);
    return noisy_proj_data

In [ ]:
noisy_data = add_noise(acquired_data,1)
plot_2d_image([1,1,1], noisy_data.as_array()[0,0,:], "Acquired data")

OK and we have some simulated data. We have added poisson noise because of photon counting statistics where we either detect a count or we don't. 

Now, back to our sensitivity image. We can either treat our sensitivity such that we correct in the projection data space as above or we can correct in our image space:
$$ \mathcal{A} = \mathcal{R} A \mathcal{N} $$

This sensitivity image will look like the backprojection of a uniform sinogram of ones

In [ ]:
sens_image = acq_model.backward(template.get_uniform_copy(1))
plot_2d_image([1,1,1], sens_image.as_array()[0,:], "sensitivity image")

Now, lets have a look at what can happen to our sensitivity image (or attenuation factors) if we have a misaligned object. \\

We'll first create a function to apply an affine transform to our image

In [ ]:
def affine_transform_2D(theta, tx, ty, sx, sy, image_arr):
    ''' create a random affine transformation for 2D images '''
    # create the transformation matrix
    transformation_matrix = np.array([[sx*np.cos(theta), -sy*np.sin(theta), tx],
                                        [sx*np.sin(theta),  sy*np.cos(theta), ty],
                                        [0, 0, 1]])

    # apply the transformation
    image_arr_transformed = affine_transform(image_arr, transformation_matrix, order=1)
    return image_arr_transformed

Now let's apply this to out attenuation image and see what happens

In [ ]:
vol_attn = attn_image.as_array()

# apply a random affine transformation using np.random
theta = np.random.uniform(-np.pi/32, np.pi/32)
tx, ty = np.random.uniform(-1, 1), np.random.uniform(-1, 1)
sx, sy = np.random.uniform(0.98, 1.08), np.random.uniform(0.98, 1.08)
vol_transformed = affine_transform_2D(theta, tx, ty, sx, sy, vol_attn[0,:])

vol_transformed = np.expand_dims(vol_transformed, axis=0)

attn_image_transform = attn_image.clone().fill(vol_transformed)

vol_im = image.as_array()
vol_transformed = affine_transform_2D(theta, tx, ty, sx, sy, vol_im[0,:])
vol_transformed = np.expand_dims(vol_transformed, axis=0)

image_transform = image.clone().fill(vol_transformed)

plt.figure()
plot_2d_image([1,2,1], attn_image.as_array()[0,:], "original attenuation image")
plot_2d_image([1,2,2], attn_image_transform.as_array()[0,:], "new attenuation image")
plt.show()

OK, so we have simulated some (large) movement

In [ ]:
acq_model_for_attn_transform = pet.AcquisitionModelUsingRayTracingMatrix() # this saves us a line of code but is the same as in the previous cell
# We now create the sensitivity model using the acquisition model and the attenuation image
asm_attn_transform = pet.AcquisitionSensitivityModel(attn_image_transform, acq_model_for_attn_transform)
asm_attn_transform.set_up(template)
attn_factors_transform = asm_attn_transform.forward(template.get_uniform_copy(1))
# And then add the detector sensitivity (based on the attenuation image) that we made previously
acq_model_transform = pet.AcquisitionModelUsingRayTracingMatrix()
acq_model_transform.set_acquisition_sensitivity(asm_attn_transform)
# set-up
acq_model_transform.set_up(template,image)
sens_image_transform = acq_model_transform.backward(template.get_uniform_copy(1))

In [ ]:
plt.figure()
plot_2d_image([1,2,1], sens_image.as_array()[0,:], "sensitivity image")
plot_2d_image([1,2,2], sens_image_transform.as_array()[0,:], "new sensitivity image")
plt.show()

Now let's compare a few reconstructions with our old and new sensitivity images. We'll use a home-made OSEM to highlight this

Firstly, let's write a quick function to deal with zero division errors outsiude of the FoV. We're using a parallel programming functionality called numba. This can be ignored. We're just setting pixels outside the FoV to zero

In [ ]:
@njit
def sensitivity_division(arr1, arr2):
    tmp  = np.zeros_like(arr1).flatten()
    for i in prange(tmp.size):
        if arr2.flatten()[i] != 0:
            tmp[i] = arr1.flatten()[i]/arr2.flatten()[i]
        else:
            tmp[i] = 0
    return tmp.reshape(arr1.shape)

Next we'll create a function to perform a step of the Maximum Likelihood Expectation Maximisation. This is a reconstruction algorithm (don;t worry too much about where it comes from) with an update equation:
$$ u_{n+1} = \frac{u_n}{A^T \mathbf{1}} A^T \frac{y}{A u_n + s + r} $$
where, as in Rahmim et al (2004) we have cancelled the attenuation image in the forward and backward steps. \\
This algorithm does the following:
1. We find the ratio of our measured data and our expected data $\frac{y}{E[y]}$
2. we back project this ratio into our image domain $A^T$
3. we multiply by the previous image iterate divided by our sensitivity image $\frac{u_n}{A^T \mathbf{1}} $
This algorithm is used regularly in clinic (most often in its subset form OSEM)

In [ ]:
# This function performs a single MLEM update
def MLEM_step(input_image, acq_model, acquired_data, sensitivity_image):
    # forward projection
    forward_projected_data = acq_model.forward(input_image)
    # divide acquired data by forward projected data
    ratio = acquired_data.divide(forward_projected_data)
    # back projection
    back_projected_data = acq_model.backward(ratio)
    # divide by sensitivity image
    back_projected_data = back_projected_data.divide(sensitivity_image)
    # update input image
    output_image = input_image*back_projected_data
    return output_image


## create initial image

We'll start with a uniform image, cropped to only fill the field of view of the scanner

In [ ]:
initial_image=image.get_uniform_copy(cmax / 4)
make_cylindrical_FOV(initial_image)
# display
im_slice = initial_image.dimensions()[0] // 2
plt.figure()
plot_2d_image([1,1,1],initial_image.as_array()[im_slice,:,:], 'initial image',[0,cmax])

In order to track how closely our estimate fits the data, we create something called an objective function. We are working with Poisson noise and so we need a data fidelity function that takes this into account. Here we use the poisson log likelihood (this comes from Poisson probability and Bayes theorem - look it up)
$$ \mathcal{D}_{PLL} = y \ln(E[y]) - E[y] $$

In [ ]:
obj_fun = pet.make_Poisson_loglikelihood(noisy_data)
obj_fun.set_acquisition_model(acq_model)
obj_fun.set_acquisition_data(noisy_data)
obj_fun.set_up(image)

As we no longer need attenuation in our acquisition model, we'll use a bog standard discrete random transform operator to do our forward and back projections

In [ ]:
radon_transform = pet.AcquisitionModelUsingRayTracingMatrix()
radon_transform.set_up(template, image)

Now let's do a reconstruction

In [ ]:
#%% run same reconstruction but saving images and objective function values every sub-iteration
num_iters = 32

# create an image object that will be updated during the iterations
current_image = initial_image.clone()

# create an array to store the values of the objective function at every
# sub-iteration (and fill in the first)
osem_objective_function_values = [obj_fun.value(current_image)]

# create an ndarray to store the images at every sub-iteration
all_osem_images = np.ndarray(shape=(num_iters + 1,) + current_image.dimensions())
all_osem_images[0,:,:,:] = current_image.as_array()

# do the actual updates
for i in range(1, num_iters+1):
    current_image = MLEM_step(current_image, radon_transform, acquired_data, sens_image)
    # store results
    obj_fun_value = obj_fun.value(current_image)
    osem_objective_function_values.append(obj_fun_value)
    all_osem_images[i,:,:,:] =  current_image.as_array()

How did we do?

In [ ]:
plot_2d_image([1,1,1], all_osem_images[-1][0,:], "reconstructed image")

## Make some plots with these results

Let's see what's going on during the reconstruction by plotting a few of the iterates along with the update and the difference between the image estimate and the ground truth

In [ ]:
#%% define a function for plotting images and the updates
def plot_progress(all_images, title, ground_truth, subiterations = [], max = cmax):
    ground_truth_array = ground_truth.as_array()
    if len(subiterations) == 0:
        num_subiters = all_images[0].shape[0] - 1
        subiterations = range(1, num_subiters + 1)
    num_rows = len(all_images)

    for i in subiterations:
        plt.figure()
        for r in range(num_rows):
            plot_2d_image([num_rows,3,3 * r + 1],
                          all_images[r][i,im_slice,:,:],'%s at %d' % (title[r], i), [0,max])
            plot_2d_image([num_rows,3,3*r+2],
                          all_images[r][i,im_slice,:,:]-all_images[r][i - 1,im_slice,:,:],'update',[-max*.05,max*.05], cmap='seismic')
            plot_2d_image([num_rows,3,3*r+3],
                          all_images[r][i,im_slice,:,:]-ground_truth_array[im_slice,:,:],'error',[-max*.5,max*.5], cmap='seismic')
        plt.show()

In [ ]:
#%% now call this function to see how we went along
# note that in the notebook interface, this might create a box with a vertical slider
subiterations = (1,2,4,8,16,24,32)
# close all "open" images as otherwise we will get warnings (the notebook interface keeps them "open" somehow)
plt.close('all')    
plot_progress([all_osem_images], ['MLEM'], image, subiterations)

And we can see what progess we make by plotting the value of our objective function for each iterate

In [ ]:
#%% plot objective function values
plt.figure()
#plt.plot(subiterations, [ osem_objective_function_values[i] for i in subiterations])
plt.plot(osem_objective_function_values)
plt.title('Objective function values')
plt.xlabel('sub-iterations');

OK. so let's now compare this to a reconstruction with the offset image, but with the original sensitivity image. We'll then compare this to a reconstruction with the correct sensitivity image

In [ ]:
noisy_data_transform = add_noise(acq_model_transform.forward(image_transform),1) # new noisy data

In [ ]:
obj_fun_transform = pet.make_Poisson_loglikelihood(noisy_data_transform)
obj_fun_transform.set_acquisition_model(acq_model_transform)
obj_fun_transform.set_acquisition_data(noisy_data_transform)
obj_fun_transform.set_up(image_transform)

Using the incorrect sensitivity

In [ ]:
#%% run same reconstruction but saving images and objective function values every sub-iteration
num_iters = 32

# create an image object that will be updated during the iterations
current_image = initial_image.clone()

# create an array to store the values of the objective function at every
# sub-iteration (and fill in the first)
osem_objective_function_values_transform = [obj_fun.value(current_image)]

# create an ndarray to store the images at every sub-iteration
all_osem_images_transform = np.ndarray(shape=(num_iters + 1,) + current_image.dimensions())
all_osem_images_transform[0,:,:,:] = current_image.as_array()

# do the actual updates
for i in range(1, num_iters+1):
    current_image = MLEM_step(current_image, radon_transform, noisy_data_transform, sens_image)
    # store results
    obj_fun_value = obj_fun_transform.value(current_image)
    osem_objective_function_values_transform.append(obj_fun_value)
    all_osem_images_transform[i,:,:,:] =  current_image.as_array()

And now using the correct sensitivity

In [ ]:
#%% run same reconstruction but saving images and objective function values every sub-iteration
num_iters = 32

# create an image object that will be updated during the iterations
current_image = initial_image.clone()

# create an array to store the values of the objective function at every
# sub-iteration (and fill in the first)
osem_objective_function_values_transform_corr = [obj_fun.value(current_image)]

# create an ndarray to store the images at every sub-iteration
all_osem_images_transform_corr = np.ndarray(shape=(num_iters + 1,) + current_image.dimensions())
all_osem_images_transform_corr[0,:,:,:] = current_image.as_array()

# do the actual updates
for i in range(1, num_iters+1):
    current_image = MLEM_step(current_image, radon_transform, noisy_data_transform, sens_image_transform)
    # store results
    obj_fun_value = obj_fun_transform.value(current_image)
    osem_objective_function_values_transform_corr.append(obj_fun_value)
    all_osem_images_transform_corr[i,:,:,:] =  current_image.as_array()

Let's see what difference a correct sensitivity made

In [ ]:
plot_2d_image([1,3,1], all_osem_images[-1][0,:], "original image", clims = [0,200])
plot_2d_image([1,3,2], all_osem_images_transform[-1][0,:], "incorr sens im", clims = [0,200])
plot_2d_image([1,3,3], all_osem_images_transform_corr[-1][0,:], "corr sens im", clims = [0,200])

This has shown us that there can be a large deviation in reconstructed image if the sensitivity image is incorrect. So how can we fix this? One method is to `gate` our acquisition into groups of timepoint with similar attenuation maps (i.e patient positions). We can then calculate a sensitivity image for all these timepoints. However, this could lead to long calculation times.

We could therefore use a neural network to output a the required change in a sensitivity image based on the change in an attenuation image.

In [ ]:
# Import a file that can generate the shepp logan phantom
from odl_funcs.ellipses import EllipsesDataset
# Import standard extra packages
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.optim as optim

mini_batch = 10
from sirf.Utilities import examples_data_path

In [ ]:
from odl_funcs.misc import *

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

class SimpleCNN(nn.Module):
    """ 
    A CNN for PET sensitivity estimation.
    Consists of 3 convolutional layers with ReLU activation. 
    Kernel sizes are 15x15, 9x9, 5x5, 3x3, 3x3, 7x7, 15x15, 3x3, 3x3 with padding 
    The idea of this is to have a large kernel size at the beginning to capture
    the large motion, and then gradually reduce the kernel size to capture the
    smaller motion.
    The kernel sizes then increases again to provide an edge filtering effect on a blurry image
    A ReLU activation is used after each convolutional layer.
    Input:
        2 images (sensitivity with no motion and attenuation correction map)
    Output:
        1 image (sensitivity with motion)
    """

    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(2, 16, 15, padding=7)
        self.conv2 = nn.Conv2d(16, 32, 9, padding=4)
        self.conv3 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv5 = nn.Conv2d(64, 32, 7, padding=3)
        self.conv6 = nn.Conv2d(32, 16, 15, padding=7)
        self.conv7 = nn.Conv2d(16, 16, 3, padding=1)
        self.conv8 = nn.Conv2d(16, 1, 3, padding=1)

    def forward(self, x):
        x[:,1,:,:]=x[:,1,:,:].mul(10)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        return x

In [ ]:
mini_batch = 2
train_dataloader = torch.utils.data.DataLoader( \
    EllipsesDataset(radon_transform, attn_image, template, mode="train", n_samples = 16) \
    , batch_size=mini_batch, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader( \
    EllipsesDataset(radon_transform, attn_image, template, mode="valid", n_samples = 4) \
    , batch_size=mini_batch, shuffle=False)

In [ ]:
loss_function = nn.MSELoss()
model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.0000001) # really small learning rate because we should be (almost) at the optimum already

In [ ]:
model.load_state_dict(torch.load("/home/sam/working/SIRF-Exercises/notebooks/PET_sensitivity/model_cnn_large.pt", map_location=device))
model.eval()
model.to(device)

In [ ]:
train_loss_history = []
valid_loss_history = []

for epoch in range(3): # 3full passes over the data
    for data, validation in zip(train_dataloader, valid_dataloader):  # `data` is a batch of data
        X, y = data  # X is the batch of features, y is the batch of targets.
        model.zero_grad()  # sets gradients to 0 before loss calc. You will do this likely every step.
        output = model(X.to(device))  # pass in the reshaped batch
        loss = loss_function(output, y.to(device))  # calc and grab the loss value
        train_loss_history.append(loss)
        #valid_loss_history.append(loss_function(net(validation[0].to(device)), validation[1].to(device)))
        loss.backward()  # apply this loss backwards thru the network's parameters
        optimizer.step()  # attempt to optimize weights to account for loss/gradients
    print(loss)  # print loss. We hope loss (a measure of wrong-ness) declines! 

In [ ]:
loss_history=[]
for i in train_loss_history:
    loss_history.append(i.item())
plt.plot(loss_history)

This was already converged, which is why you see this flat, noisy line here. The training objective function looked more like this (note we're minimising the objective function for the network - this is just a converntion thing)

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('/home/sam/working/SIRF-Exercises/notebooks/PET_sensitivity/loss_history_CNN_large.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
input_data = torch.from_numpy(np.squeeze(np.array([sens_image.as_array(), attn_image_transform.as_array()]))).unsqueeze(0)
input_data = input_data.to(device)

In [ ]:
out = model(input_data)

In [ ]:
sens_image_NN = sens_image.clone().fill(out.detach().cpu().numpy()[0,:,:,:])

In [ ]:
plot_2d_image([1,3,1], sens_image.as_array()[0,:,:], 'sens im', cmap="viridis")
plot_2d_image([1,3,2], sens_image_transform.as_array()[0,:,:], 'sens im trans', cmap="viridis")
plot_2d_image([1,3,3], sens_image_NN.as_array()[0,:,:], 'sens im corr', cmap="viridis")

And that was surprisingly good! There's definitely a lot wrong, but you can se the network was trying to do what we asked - how could we improve this network to more accurately predict out sensitivity image?

Let's see how this looks when we reconstruct it

In [ ]:
#%% run same reconstruction but saving images and objective function values every sub-iteration
num_iters = 32

# create an image object that will be updated during the iterations
current_image_NN = initial_image.clone()

# create an array to store the values of the objective function at every
# sub-iteration (and fill in the first)
osem_objective_function_values_NN = [obj_fun.value(current_image_NN)]

# create an ndarray to store the images at every sub-iteration
all_osem_images_NN = np.ndarray(shape=(num_iters + 1,) + current_image_NN.dimensions())
all_osem_images_NN[0,:,:,:] = current_image_NN.as_array()

# do the actual updates
for i in range(1, num_iters+1):
    current_image_NN = MLEM_step(current_image_NN, radon_transform, noisy_data_transform, sens_image_NN)
    # store results
    all_osem_images_NN[i,:,:,:] = current_image_NN.as_array()

In [ ]:
plot_2d_image([2,3,1], all_osem_images_NN[-1,0,:,:], 'Neural Net', cmap="viridis", clims=[0,150])
plot_2d_image([2,3,2], all_osem_images_transform[-1,0,:,:], 'incorrect map', cmap="viridis", clims=[0,150])
plot_2d_image([2,3,3], all_osem_images_transform_corr[-1,0,:,:], 'Gold Standard', cmap="viridis", clims=[0,150])
plot_2d_image([2,3,4], all_osem_images_NN[-1,0,:,:]-all_osem_images_transform_corr[-1,0,:,:], 'diff', cmap="viridis")
plot_2d_image([2,3,5], all_osem_images_transform[-1,0,:,:]-all_osem_images_transform_corr[-1,0,:,:], 'diff', cmap="viridis")
plot_2d_image([2,3,6], all_osem_images_transform_corr[-1,0,:,:]-all_osem_images_transform_corr[-1,0,:,:], 'diff', cmap="viridis")

And it's not great, but this is just a start!

# Things to think about
## What network architecture is best? A UNet, maybe?
## We could try using difference images. How would we need to change the network (would a ReLu work?)
## Is the the best loss function (SSIM, MAE, L1 loss, smoothed loss)
## Will more training data fix our issue?
## Are ellipses really the best option for training (Brainweb training data?)
## Can we simulate non-affine transforms (yawning etc.)

### And many more...